<a href="https://colab.research.google.com/github/CHOISEONGGU/MachineLearning_ChoiSG/blob/main/CHOISEONGGU/%EA%B3%B5%EB%B6%80%EC%9E%90%EB%A3%8C/CH08_08%2B09%2B10_%EB%B3%B5%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LightGBM

## 전처리

### 명목형
* Object Dtype -> int, float (numeric) 숫자형의 변수형 (명목형 -> 순서형, 이산형, 연속형...)
    * 머신러닝 모델들은 숫자형태의 데이터만 사용가능
    * 머신러닝 모델링 작업은 비교하거나 통계적 계산을 통해서 공식화 혹은 기준화

> **라벨 인코딩** : 순서가 있거나 혹은 알고리즘 자체가 숫자를 가지고 비교연산만 할 때 (항목들이 있으면 해당 항목의 고윳값들을 숫자로 치환 - 주로 자연수로) ex) 초, 중, 고 -> 1, 2, 3. => 순서가 없거나, 알고리즘 자체가 통계적 연산을 필요로 할 때 잘못된 결과를 가져올 수 있음 (선형 기반, 거리 기반)

> **원핫 인코딩 (더미변수)** : 순서와는 상관없이, 해당 속성을 보유하고 있는지(1) 아닌지 (0) 를 기준으로 분류하는 방법. -> 소속, 속성, 특성. => 해당 열의 고윳값의 수에 따라 더미변수의 수가 급격하게 늘어날 수 있음. (1개씩 줄일 순 있는데 큰 영향은 없음), 특정 '열(속성)'의 영향도를 한 번에 파악하기 어려움

* 겉으로는 숫자 단위처럼 되어 있는데 dtype object인 경우 -> astype(int) 혹은 astype(float)
    * `,`나 `단위`가 섞여 있는 경우 -> split으로 분리하거나, replace로 변환해서 사용, 정규표현식(regular expression) 사용해서 처리 (못하겠으면 챗선생한테 문의)

### 결측치
* [missingno](https://hong-yp-ml-records.tistory.com/14)
* df.info, df.isna().mean(), sum()... -> plot() /

* 해당 열의 결측치를 가진 행이 10% 미만
    * dropna, dropna(subset=...)
    * 평균치나, 최빈값으로 채워주기 (통계값에 큰 영향을 주지 않기 때문에...)
        * groupby 같은 걸로 해서, 중요하다고 생각하는...
        * 지도학습의 경우에는? y값과의 상관성이 높은 변수들...? 도메인 지식...
    * (결측치 표시)

* 애매할 때? -> 20~30%.
    * groupby... -> 그 그룹간의 집합연산을 통해서 해당 값을 채우거나
    * 트리 모델의 경우에는 해당 전처리에서는 사용될 일 없는, 데이터가 겹칠 일이 없는 -99, -1 같은 변수를 사용해서 '결측치'라는 걸 표현

* 장난 아닐 때 -> 70~80%
    * (결측치라는 걸 나타내는 방법)
    * 열을 버린다 (axis=1 drop)

* 결측치 구분은 '트리 모델'일 때만, 거리 기반이나 선형 기반은 쓸 수 X

## 피쳐 엔지니어링
> 정확도를 높이거나, 오류를 줄이기 위해 하는 변수 합성 혹은 수정

### 피쳐 셀렉션
* 영향도가 없거나 오히려 방해하는 것 같은 경우 drop하는 것
    * `feature_importances_` : 변수가 얼마나 앙상블한 모델에서 선택되었나? 영향을 주었냐? -> 이런 영향이 거의 없는 변수는 없는게 낫다
* 다중공산성 -> 각각 관측값(독립변수) -> 서로간의 독립 -> 안되어 있다면? -> 둘 중에 하나를 drop하거나, 한 차원으로 합성 -> `PCA`로 합성

### 변수를 만드는 경우
* 명목형 변수 -> 일부를 추출해서 해당 변수를 또다른 그룹 -> 타이타닉에서 `title` (도메인 지식 -> 산업이나 영역에 대해서의 지식이 중요)
* 단위나 다른 두 개 이상의 변수를 활용해서 환산한 값을 만들어냄 -> 중고차에서 연비 (연료 종류 + 마일리지 -> km 대비 가격으로 환산)
* 통계(z점수, 평균, 최고-최저 차이, 변화율)값들을 활용해서 특정한 그룹 혹은 속성을 지닌 값들에 대해서 추가적인 변수를 만들 수 있음

## 평가 (이진분류)

* 재현율(recall), 정밀도(precision), f1점수(f1-score) -> 분류 리포트
* 0.5라는 결과값(기대값)을 기준으로 0, 1인지를 판단 -> 이 기준이 적절한가?
* 0.5라는 기준을 움직임에 따라서, 1종오류(0->1 위양성), 2종오류(1->0 위음성) 중에 무엇을 중요시 여기냐에 따라 재,정,f1 모두 변화 가능
    * 기준을 느슨하게 하면? -> 1종 오류가 늚
    * 기준을 엄격하게 하면? -> 2종 오류가 늚

### ROC-AUC
![ROC](https://i.imgur.com/euCumVh.png)
* 이진분류 모델을 평가하는 방법으로 기준점에 영향을 받지 않기 때문에 여러 모델을 비교할 때 요긴하게 사용
* AUC는 ROC 곡선의 아래 면적을 의미
    * 0.5~1 사이의 값을 지니며 높을수록 좋은 모델

## ROC 곡선
> 민감도, 특이도 개념을 활용

### 민감도 (TPR)
$TPR = \frac{TP(참 양성)}{TP(참 양성) + FN(거짓 음성)}$
* 재현율과 수식이 같음
* 실제 1인 것 중 얼마만큼 제대로(1로) 예측되었는지
* 1에 가까울 수록 좋은 수치

### 특이도 (TNR) **
$TNR = \frac{TN(참 음성)}{FP(거짓 양성) + TN(참 음성)}$
* 실제 0인 것 중 얼마만큼 제대로(0로) 예측되었는지
* 1에 가까울 수록 좋은 수치

$FPR = \frac{FP(거짓 양성)}{FP(거짓 양성) + TN(참 음성)}$
* 실제 0인 것 중 얼마만큼 잘못(1로) 예측되었는지
* 0에 가까울 수록 좋은 수치
$FPR = 1 - TNR$

민감도(TPR)을 높이기 위해서 기준을 낮추면, 반대로 FPR은 증가하게 되어 있음
(역의 관계)
하지만 예측력이 좋은 모델이라면 그 변화치가 높지 않음

![ROC](https://i.imgur.com/euCumVh.png)
* 기준점을 바꿨을 때 TPR, FPR이 어떻게 바뀌는지 보여주는 그래프
* 최악의 경우 (학습이 전혀 안될 경우) -> 빨간색 점선

### AUC (Area Under the ROC Curve)
* ROC 곡선의 아래쪽에 해당하는 면적
* 0.5~1 사이의 값을 가지며 커질 수록 더 좋은 분류기라는 의미

![AUC](https://i.imgur.com/udlCMW4.png)

## 하이퍼 패러미터 튜닝
* 랜덤 서치 : 그리드 서치 좋은데 너무 오래 걸려 -> 몇몇 조합으로 (랜덤하게)만들어보고 그 중에서 좋았던 패러미터들을 가지고 비교해주는 방식

# 비지도 학습

## K-평균 군집화
* K : 내가 K를 정해줘야한다 (K-Nearest Neighbor, K-fold)
* 내가 데이터를 k개의 그룹(클러스터)로 묶어주겠다 -> '거리'를 기반으로 
* 스케일링 -> 표준화 스케일링 (로버스트 스케일링, 최대최소 스케일링...)
* 라벨값이 없을 때 적절한 결과값을 도출하기 위해
* 피처 엔지니어링할 때 -> 데이터간의 적절한 상관성(거리)을 바탕으로 새로운 변수 만들기
* 시각화할 때 hue class?
### 적절한 K 값 어떻게 구해줄까?
* 엘보우 기법 (elbow) : 팔꿈치 -> 특정한 수치(이너셔: 클러스터의 중심부터 각 분류된 요소들의 거리 평균)가 팍 꺾이는 부분을 분기점으로 해서 해당 k값을 선택해주는 방법
    * 잘 안보이면? 너무 부드럽게 증감하면?
* 실루엣 계수 : 중심으로부터 각 값의 거리 + 클러스터 간 거리 -> 클수록 좋음

### PCA (Principal Component Analysis: 주성분 분석)
* 변환 알고리즘
* 차원을 축소 (여러 개의 변수를 적은 갯수의 변수로 나타내는 기법)
1. 시각화 (2~3차원) -> 여러 개의 변수가 있을 경우에 해당 변수의 분포를 (y값에 따른) 2차원으로 나타내주고 싶다면 변수를 2차원으로 축소해서 시각화
1. 독립변수가 너무 많을 때, 학습속도를 높이거나 과최적화 방지를 위해서 PCA를 통해 적절한 수준의 차원 축소 (70~80% 정도의 설명력을 남기고)
* 적당한 축소 수준을 정하는게 중요
    * `explained_variance_ratio_` <- 엘보
